# Modeling Notebook


## 1. Import libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os 
%config Completer.use_jedi = False

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
os.sys.path.append('../src')

import experiments


## 2. Load Data

In [3]:
# df_raw = pd.read_excel('../data/survey_answers.xlsx', sheet_name='Dados_brutos')
df_raw = pd.read_csv('../data/survey-answers-no-outliers.txt', encoding = 'iso 8859-1', delimiter = '\t')
df_raw.head()

,Genero,Idade,Escolaridade,Estado_Civil,Filhos,Atividade_fisica_semana,Cigarros_dia,Horas_Dormidas_dia,Se_Capacitando,Outro_emprego,Ambiente_Trabalho,Categoria_Profissional,Tempo_Empresa_anos,Tempo_Funcao_anos,Horas_Trabalhadas_Semana,Pausas_Programadas_minutos,Trabalho_Entre_Ferias_meses,Recebeu_Treinamento,Como_foi_treinado,Trabalho_Chato,Trabalho_Organizado,Prazos_Apertados,Satisfacao_Gerencia,Satisfacao_Dificuldades,Tempo_Pe_horas,Tempo_Sentado_horas,Tempo_Corcoras_horas,Superiores_Desconfortavel_horas,Inferiores_Desconfortavel_horas,Tronco_Curvado_horas,Tronco_Torcido_horas,Maos_Dedos_horas,Movimentos_repetitivos_horas,Movimentos_Rapidos_horas,Ferramentas_Maos_horas,Ferramentas_Corpo_horas,dados.Costa_superior,dados.Costa_media,dados.Costa_inferior,bmi,carry_weight
0,1,36,3,2,2,0,0,6,2,2,2,1,8.0,3.0,36,30,11,2,1,1,5,5,5,4,11.0,0.5,0.0,0.0,0.0,1.0,0.0,12.0,12.0,12.0,1.0,0,1,1,1,15.418265,3.0
1,1,30,3,2,1,5,0,6,1,2,2,1,4.0,7.0,40,120,11,1,5,1,4,1,4,3,4.0,4.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0,1,2,1,16.765737,0.0
2,1,40,3,2,3,0,0,10,1,2,1,1,8.0,12.0,36,60,11,1,4,1,5,1,4,3,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,3.0,0.0,0,2,1,1,19.297399,0.0
3,1,45,3,2,1,3,0,8,1,2,1,1,8.0,18.0,36,60,11,1,5,2,5,4,4,3,6.0,6.0,0.0,4.0,8.0,2.0,0.0,12.0,12.0,3.0,0.0,0,1,1,1,17.578125,0.0
4,1,41,5,1,2,5,0,7,2,2,1,1,4.0,4.0,36,0,11,1,3,1,5,3,4,4,11.0,1.0,1.0,10.0,10.0,0.0,0.0,12.0,12.0,10.0,0.0,0,1,1,1,15.943878,3.0


In [4]:
continous_columns = [
                    'Idade', 
                     'bmi',
                     'Filhos',
                     'Atividade_fisica_semana',
                     'Cigarros_dia',
                     'Horas_Dormidas_dia',
                     'Tempo_Empresa_anos',
                    #'Horas_Trabalhadas_Dia',
                    'Tempo_Funcao_anos',
                     'Horas_Trabalhadas_Semana',
                     'Pausas_Programadas_minutos',
                    'Trabalho_Entre_Ferias_meses',
                    'Tempo_Pe_horas',
                    'Tempo_Sentado_horas',
                    'Tempo_Corcoras_horas',
                    'Superiores_Desconfortavel_horas',
                    'Inferiores_Desconfortavel_horas',
                    'Tronco_Curvado_horas',
                    'Tronco_Torcido_horas',
                    'Maos_Dedos_horas',
                    'carry_weight',
                    'Movimentos_repetitivos_horas',
                    'Movimentos_Rapidos_horas',
                    'Ferramentas_Maos_horas',
                    'Ferramentas_Corpo_horas'
                    ]
categorical_columns = ['Genero',
                       'Estado_Civil',
                       'Se_Capacitando',
                       'Outro_emprego',
                       'Ambiente_Trabalho',
                       'Categoria_Profissional',
                       'Recebeu_Treinamento',
                       'Como_foi_treinado',
                       'Escolaridade',
                       'Trabalho_Chato',
                       'Trabalho_Organizado',
                       'Prazos_Apertados',
                       'Satisfacao_Gerencia',
                       'Satisfacao_Dificuldades'
                       ]
target_columns = ['dados.Costa_superior', 'dados.Costa_media', 'dados.Costa_inferior']

useful_columns = continous_columns + categorical_columns + target_columns

In [5]:
df_raw.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Genero                           398 non-null    int64  
 1   Idade                            398 non-null    int64  
 2   Escolaridade                     398 non-null    int64  
 3   Estado_Civil                     398 non-null    int64  
 4   Filhos                           398 non-null    int64  
 5   Atividade_fisica_semana          398 non-null    int64  
 6   Cigarros_dia                     398 non-null    int64  
 7   Horas_Dormidas_dia               398 non-null    int64  
 8   Se_Capacitando                   398 non-null    int64  
 9   Outro_emprego                    398 non-null    int64  
 10  Ambiente_Trabalho                398 non-null    int64  
 11  Categoria_Profissional           398 non-null    int64  
 12  Tempo_Empresa_anos    

## 3. Feature Engineering

In [6]:
df_analysis = df_raw[useful_columns].copy()
# df_analysis['IMC'] = df_analysis['Peso']/df_analysis['Altura']**2

In [7]:
def bmi_category(bmi_value):
    if bmi_value < 18:
        return 'underweight'
    elif bmi_value >= 18 and bmi_value <= 24:
        return 'normal'
    elif bmi_value > 24 and bmi_value <= 28:
        return 'overweight'
    elif bmi_value > 28:
        return 'obese'
    else:
        return 'error'

In [8]:
df_analysis.insert(loc = 1, column = 'bmi_category', value = df_analysis['bmi'].apply(bmi_category))

In [9]:
# df_analysis.insert(loc = 1, 
#                    column = 'Horas_Carregando_Carga', 
#                    value = df_analysis['Carga_6_horas'] + df_analysis['Carga_15_horas'] + df_analysis['Carga_25_horas'])

In [10]:
df_analysis[target_columns[0]].value_counts()

1    177
2    158
3     63
Name: dados.Costa_superior, dtype: int64

In [11]:
df_analysis[target_columns[1]].value_counts()

1    187
2    144
3     67
Name: dados.Costa_media, dtype: int64

In [12]:
df_analysis[target_columns[2]].value_counts(normalize = True)

1    0.427136
2    0.389447
3    0.183417
Name: dados.Costa_inferior, dtype: float64

## 4. Target definition

In [13]:
def target_grouping(score):
    if score >= 4:
        return 'severe pain'
    elif score in [2,3]:
        return 'mild pain'
    else:
        return 'no pain'

In [14]:
# df_analysis['Costa_superior_agg'] = df_analysis['Costa_superior'].apply(target_grouping)
# df_analysis['Costa_media_agg'] = df_analysis['Costa_media'].apply(target_grouping)
# df_analysis['Costa_inferior_agg'] = df_analysis['Costa_inferior'].apply(target_grouping)

## 5. Feature Selection

We'll fit a Random Forest model and will use its feature importance values to select the most relevant features.

- [Article](https://johaupt.github.io/blog/columnTransformer_feature_names.html) with wraper function to get feature names

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df_analysis.drop(columns = target_columns)
y = df_analysis[target_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


preprocessing = experiments.first_preprocessing
model = DecisionTreeClassifier(random_state = 42)

sel = SelectFromModel(model)
X_train_preprocessed = preprocessing.fit_transform(X_train)
sel = model.fit(X_train_preprocessed, y_train[target_columns[0]])

In [44]:
import sklearn
import warnings
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [45]:
feature_importance_df = pd.DataFrame({'feature_name': get_feature_names(preprocessing),
                                      'feature_importance': np.round(sel.feature_importances_, 4)
                                     })\
                        .sort_values(by =['feature_importance'], ascending = False)
feature_importance_df.head(18)

/var/folders/b7/bgdywzjn66x2_2jj0sl00131cxdxqv/T/ipykernel_88038/733556529.py:32: UserWarning: Transformer imputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
/var/folders/b7/bgdywzjn66x2_2jj0sl00131cxdxqv/T/ipykernel_88038/733556529.py:32: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


,feature_name,feature_importance
6,num__Pausas_Programadas_minutos,0.1214
11,num__Tempo_Empresa_anos,0.0910
2,num__Tempo_Funcao_anos,0.0897
4,num__Tempo_Sentado_horas,0.0663
3,num__bmi,0.0649
9,num__Trabalho_Entre_Ferias_meses,0.0621
20,encoder__x2_2,0.0435
7,num__Inferiores_Desconfortavel_horas,0.0428
8,num__Tempo_Pe_horas,0.0415
0,num__Movimentos_Rapidos_horas,0.0413


In [46]:
feature_importance_df.head(18)['feature_name'].tolist()

['num__Pausas_Programadas_minutos',
 'num__Tempo_Empresa_anos',
 'num__Tempo_Funcao_anos',
 'num__Tempo_Sentado_horas',
 'num__bmi',
 'num__Trabalho_Entre_Ferias_meses',
 'encoder__x2_2',
 'num__Inferiores_Desconfortavel_horas',
 'num__Tempo_Pe_horas',
 'num__Movimentos_Rapidos_horas',
 'num__Idade',
 'encoder__x4_6',
 'encoder__x0_3',
 'encoder__x3_3',
 'num__Horas_Trabalhadas_Semana',
 'encoder__x0_2',
 'encoder__x1_1',
 'num__carry_weight']

In [27]:
feature_importance_df.head(18)['feature_importance'].sum()

0.8949

In [52]:
experiments.categorical_columns[1]

'Estado_Civil'

In [28]:
selected_features = [
    'Horas_Trabalhadas_Semana',
     'Tempo_Funcao_anos',
     'Movimentos_Rapidos_horas',
     'Satisfacao_Gerencia',
     'Tempo_Empresa_anos',
     'Tempo_Sentado_horas',
     'Tempo_Pe_horas',
     'Escolaridade',
     'Trabalho_Entre_Ferias_meses',
     'carry_weight',
     'Inferiores_Desconfortavel_horas',
     'Idade',
     'Prazos_Apertados',
     'Pausas_Programadas_minutos',
     'bmi',
     'Recebeu_Treinamento',
     'Trabalho_Chato'
]

In [30]:
selected_categorical = experiments.selected_categorical
selected_continous = experiments.selected_continous

In [31]:
print(selected_categorical)
print(selected_continous)

['Trabalho_Chato', 'Recebeu_Treinamento', 'Satisfacao_Gerencia', 'Prazos_Apertados', 'Escolaridade']
['Movimentos_Rapidos_horas', 'carry_weight', 'Tempo_Funcao_anos', 'bmi', 'Tempo_Sentado_horas', 'Horas_Trabalhadas_Semana', 'Pausas_Programadas_minutos', 'Inferiores_Desconfortavel_horas', 'Tempo_Pe_horas', 'Trabalho_Entre_Ferias_meses', 'Idade', 'Tempo_Empresa_anos']


## 6. Train and test split
We'll split the data into train and test set. The train set will be used to perform 10-fold cross validation. We'll compute metrics for the test set and compare them.

In [32]:
from sklearn.model_selection import train_test_split
# target_columns = ['Costa_superior_agg', 'Costa_media_agg', 'Costa_inferior_agg']
target_columns = ['dados.Costa_superior', 'dados.Costa_media', 'dados.Costa_inferior']

X = df_analysis[selected_continous + selected_categorical]
y = df_analysis[target_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## 7. Experiments

- [Article](https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0) used as reference to wrap code.

### 7.1. Upper Back

In [34]:
scorer = experiments.scorers
algorithms = experiments.build_algorithms()
cv = experiments.cv
target_column = target_columns[0]

In [35]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate

def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
    dfs = []

    # results = []
    names = []
    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
    target_names = ['no pain', 'severe pain', 'mild pain']
    for name, model in algorithms.items():
        kfold = cv#StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
        cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        # results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
    final = pd.concat(dfs, ignore_index=True)
    return final

In [36]:
run_exps(X_train, y_train[target_column], X_test, y_test[target_column])

kNN
              precision    recall  f1-score   support

     no pain       0.59      0.57      0.58        53
 severe pain       0.47      0.57      0.52        47
   mild pain       0.25      0.15      0.19        20

    accuracy                           0.50       120
   macro avg       0.44      0.43      0.43       120
weighted avg       0.49      0.50      0.49       120

tree
              precision    recall  f1-score   support

     no pain       0.44      0.66      0.53        53
 severe pain       0.38      0.30      0.33        47
   mild pain       0.00      0.00      0.00        20

    accuracy                           0.41       120
   macro avg       0.27      0.32      0.29       120
weighted avg       0.34      0.41      0.36       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svmlinear
              precision    recall  f1-score   support

     no pain       0.46      0.70      0.55        53
 severe pain       0.44      0.36      0.40        47
   mild pain       0.00      0.00      0.00        20

    accuracy                           0.45       120
   macro avg       0.30      0.35      0.32       120
weighted avg       0.37      0.45      0.40       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/an

ann
              precision    recall  f1-score   support

     no pain       0.52      0.62      0.57        53
 severe pain       0.47      0.47      0.47        47
   mild pain       0.20      0.10      0.13        20

    accuracy                           0.48       120
   macro avg       0.40      0.40      0.39       120
weighted avg       0.45      0.47      0.46       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model
0,0.490207,0.008177,0.430108,0.385570,0.379785,0.381632,kNN
1,0.454853,0.008172,0.569892,0.600000,0.470613,0.469808,kNN
2,0.439250,0.008029,0.489130,0.389525,0.398233,0.392857,kNN
3,0.415552,0.005140,0.408602,0.334776,0.338856,0.331062,tree
4,0.390004,0.004692,0.516129,0.673913,0.419348,0.410089,tree
5,0.466737,0.006069,0.445652,0.379251,0.379634,0.379038,tree
6,0.500630,0.010489,0.483871,0.313636,0.378159,0.337954,svmlinear
7,0.509252,0.006443,0.483871,0.316550,0.373230,0.338865,svmlinear
8,0.527904,0.006395,0.478261,0.332026,0.365634,0.312261,svmlinear
9,1.993480,0.006550,0.344086,0.292330,0.296257,0.291464,ann


### 7.2. Mid Back

In [38]:
target_column = target_columns[1]
run_exps(X_train, y_train[target_column], X_test, y_test[target_column])

kNN
              precision    recall  f1-score   support

     no pain       0.45      0.65      0.53        52
 severe pain       0.49      0.37      0.42        51
   mild pain       0.20      0.06      0.09        17

    accuracy                           0.45       120
   macro avg       0.38      0.36      0.35       120
weighted avg       0.43      0.45      0.42       120

tree
              precision    recall  f1-score   support

     no pain       0.58      0.50      0.54        52
 severe pain       0.52      0.67      0.58        51
   mild pain       0.33      0.18      0.23        17

    accuracy                           0.53       120
   macro avg       0.48      0.45      0.45       120
weighted avg       0.52      0.53      0.51       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svmlinear
              precision    recall  f1-score   support

     no pain       0.43      1.00      0.60        52
 severe pain       0.00      0.00      0.00        51
   mild pain       0.00      0.00      0.00        17

    accuracy                           0.43       120
   macro avg       0.14      0.33      0.20       120
weighted avg       0.19      0.43      0.26       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/an

ann
              precision    recall  f1-score   support

     no pain       0.47      0.62      0.53        52
 severe pain       0.44      0.35      0.39        51
   mild pain       0.18      0.12      0.14        17

    accuracy                           0.43       120
   macro avg       0.36      0.36      0.36       120
weighted avg       0.42      0.43      0.42       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model
0,0.468847,0.007938,0.397849,0.255385,0.300836,0.270996,kNN
1,0.440057,0.008154,0.537634,0.575240,0.434915,0.429101,kNN
2,0.448013,0.008576,0.467391,0.403371,0.395594,0.395370,kNN
3,0.451741,0.005205,0.440860,0.367308,0.346012,0.328716,tree
4,0.422382,0.006197,0.365591,0.328112,0.327416,0.327313,tree
5,0.445032,0.005492,0.402174,0.350213,0.337724,0.341902,tree
6,0.517341,0.007504,0.430108,0.272222,0.333094,0.298710,svmlinear
7,0.504250,0.006796,0.483871,0.161290,0.333333,0.217391,svmlinear
8,0.533641,0.008135,0.489130,0.163043,0.333333,0.218978,svmlinear
9,1.958357,0.006465,0.440860,0.405187,0.401504,0.402609,ann


### 7.3. Low Back

In [39]:
target_column = target_columns[2]
run_exps(X_train, y_train[target_column], X_test, y_test[target_column])

kNN
              precision    recall  f1-score   support

     no pain       0.50      0.55      0.52        51
 severe pain       0.47      0.46      0.46        46
   mild pain       0.37      0.30      0.33        23

    accuracy                           0.47       120
   macro avg       0.45      0.44      0.44       120
weighted avg       0.46      0.47      0.46       120

tree
              precision    recall  f1-score   support

     no pain       0.56      0.53      0.55        51
 severe pain       0.40      0.54      0.46        46
   mild pain       0.11      0.04      0.06        23

    accuracy                           0.44       120
   macro avg       0.36      0.37      0.36       120
weighted avg       0.41      0.44      0.42       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svmlinear
              precision    recall  f1-score   support

     no pain       0.59      0.67      0.62        51
 severe pain       0.48      0.65      0.56        46
   mild pain       0.00      0.00      0.00        23

    accuracy                           0.53       120
   macro avg       0.36      0.44      0.39       120
weighted avg       0.43      0.53      0.48       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/56325/opt/an

ann
              precision    recall  f1-score   support

     no pain       0.50      0.59      0.54        51
 severe pain       0.47      0.41      0.44        46
   mild pain       0.50      0.43      0.47        23

    accuracy                           0.49       120
   macro avg       0.49      0.48      0.48       120
weighted avg       0.49      0.49      0.49       120



/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model
0,0.467860,0.008113,0.430108,0.373137,0.356644,0.344328,kNN
1,0.454706,0.007958,0.451613,0.426935,0.421187,0.422133,kNN
2,0.438066,0.008331,0.456522,0.455983,0.399279,0.389770,kNN
3,0.409755,0.004845,0.397849,0.378382,0.391441,0.380549,tree
4,0.390971,0.004746,0.397849,0.330580,0.335349,0.324662,tree
5,0.384682,0.004854,0.391304,0.424183,0.354408,0.358677,tree
6,0.490905,0.006287,0.408602,0.276803,0.331532,0.296379,svmlinear
7,0.481979,0.006497,0.430108,0.143369,0.333333,0.200501,svmlinear
8,0.524467,0.007778,0.456522,0.301720,0.370370,0.330516,svmlinear
9,1.918548,0.006192,0.344086,0.320244,0.315653,0.312241,ann


## 8. Feature Analysis